In [4]:
import pandas as pd
 

In [5]:
data_train=pd.read_csv('0000000000002747_training_twitter_x_y_train.csv')

In [6]:
X_train=list(data_train['text'])

In [7]:
X_train

['@SouthwestAir I am scheduled for the morning, 2 days after the fact, yes..not sure why my evening flight was the only one Cancelled Flightled',
 '@SouthwestAir seeing your workers time in and time out going above and beyond is why I love flying with you guys. Thank you!',
 '@united Flew ORD to Miami and back and  had great crew, service on both legs. THANKS',
 "@SouthwestAir @dultch97 that's horse radish 😤🐴",
 '@united so our flight into ORD was delayed because of Air Force One, but the last flight to SBN is at 8:20, 5 mins from now we just landed.',
 '@united Why did you load us in this flying sardine can if you knew the pilots were 2 hours Late Flight?? #incompetent beyond belief',
 "@JetBlue that is a stock response. Delays not as frustrating as poor cust serv &amp; being told by 3 ppl to wait &amp; they'd come back but did not.",
 "@JetBlue That'd be nice! Hoping to rack up enough miles to take a trip to Seattle and enjoy a perfect latte in the city of coffee.",
 '@united frankly

In [8]:
Y_train=list(data_train['airline_sentiment'])

In [9]:
X_test=pd.read_csv('0000000000002747_test_twitter_x_test.csv')

In [10]:
X_test=list(X_test['text'])


In [11]:
combined_tweet=X_train+X_test

In [14]:

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
all_words = [word_tokenize(tweet) for tweet in combined_tweet]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [15]:
all_words

[['@',
  'SouthwestAir',
  'I',
  'am',
  'scheduled',
  'for',
  'the',
  'morning',
  ',',
  '2',
  'days',
  'after',
  'the',
  'fact',
  ',',
  'yes',
  '..',
  'not',
  'sure',
  'why',
  'my',
  'evening',
  'flight',
  'was',
  'the',
  'only',
  'one',
  'Cancelled',
  'Flightled'],
 ['@',
  'SouthwestAir',
  'seeing',
  'your',
  'workers',
  'time',
  'in',
  'and',
  'time',
  'out',
  'going',
  'above',
  'and',
  'beyond',
  'is',
  'why',
  'I',
  'love',
  'flying',
  'with',
  'you',
  'guys',
  '.',
  'Thank',
  'you',
  '!'],
 ['@',
  'united',
  'Flew',
  'ORD',
  'to',
  'Miami',
  'and',
  'back',
  'and',
  'had',
  'great',
  'crew',
  ',',
  'service',
  'on',
  'both',
  'legs',
  '.',
  'THANKS'],
 ['@', 'SouthwestAir', '@', 'dultch97', 'that', "'s", 'horse', 'radish', '😤🐴'],
 ['@',
  'united',
  'so',
  'our',
  'flight',
  'into',
  'ORD',
  'was',
  'delayed',
  'because',
  'of',
  'Air',
  'Force',
  'One',
  ',',
  'but',
  'the',
  'last',
  'flight',

In [16]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [18]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
stops=(stopwords.words('english'))
punctuations=list(string.punctuation)
stops = stops + punctuations

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [19]:
def clean_review(words):
    output_words=[]
    for w in words:
        if w.lower() not in stops and w.lower()[0] != '/' and w.lower() != 'https' and w.lower() != 'htpp':
            pos=pos_tag([w])
            clean_word=lemmatizer.lemmatize(w,pos=get_simple_pos(pos[0][1]))#as it is stored in 0th and then 1st [('better', 'RBR')]
            output_words.append(clean_word.lower())
    return output_words

In [20]:
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [23]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [25]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [27]:
nltk.download('omw-1.4')


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [28]:
clean_tweets=[(clean_review(tweets)) for tweets in all_words]

In [29]:
text_tweets=[" ".join(tweet) for tweet in clean_tweets  ]

In [30]:
X_train_usable_tweets = text_tweets[0:10980]

In [31]:
X_test_usable_tweets = text_tweets[10980:]

In [32]:
from sklearn.feature_extraction.text import CountVectorizer

In [33]:
count_vec = CountVectorizer(max_features=2000,min_df=0.002,max_df=0.95)
X_train_features=count_vec.fit_transform(X_train_usable_tweets)


In [34]:
len(count_vec.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


721

In [35]:
X_test_features = count_vec.transform(X_test_usable_tweets)

In [36]:
X_test_features

<3660x721 sparse matrix of type '<class 'numpy.int64'>'
	with 27229 stored elements in Compressed Sparse Row format>

In [37]:
from sklearn.svm import SVC

In [38]:
svc = SVC()
svc.fit(X_train_features ,Y_train)

SVC()

In [39]:
data_pred=svc.predict(X_test_features)

In [40]:
df=pd.DataFrame(data_pred)


In [41]:
df.to_csv("predictionsbyhp.csv",header=False,index=False)

In [42]:
from sklearn.ensemble import RandomForestClassifier

In [43]:
clf = RandomForestClassifier(n_estimators=500)
clf.fit(X_train_features,Y_train)

RandomForestClassifier(n_estimators=500)

In [44]:
y_pred = clf.predict(X_test_features)

In [45]:
len(y_pred)

3660

In [47]:
import numpy as np

In [48]:
np.savetxt("predbyme.csv",y_pred, delimiter=',', fmt="%s")